In [ ]:
from sklearn.naive_bayes import CategoricalNB
import json
import seaborn as sns
import pandas as pd


In [ ]:
with open('masterchef_data_cleaned.json', 'r') as file:
    data = json.load(file)

In [ ]:
data.keys()

In [ ]:
# data.keys()
US = pd.DataFrame(data['United States'])
Canada = pd.DataFrame(data['Canada'])
Brazil = pd.DataFrame(data['Brazil'])


In [ ]:
df = pd.concat([US, Canada, Brazil])

In [ ]:
def winner(row):
    return row['evaluations'][-1] == 'WINNER'



In [ ]:
pd.DataFrame(df[(df['season'] == 1) & (df['nation'] == 'United States')]['evaluations'])
new_df = df.copy()
new_df

In [ ]:
new_df['won'] = new_df.apply(winner,axis=1)

In [ ]:
winners = new_df[new_df['won']]

# Function to generate season ID
def generate_season_id(row):
    country_id = {"USA": 1, "Canada": 2, "Brazil": 3}
    
    season_id = row['nation'] + str(row['season'])
    
    return season_id




new_df['seasonID'] = new_df.apply(generate_season_id, axis=1)
winners['seasonID'] = winners.apply(generate_season_id, axis=1)



In [ ]:
new_df['seasonID'].unique()

In [ ]:
winners['seasonID'].unique()

In [ ]:
new_df['age'].mean()

In [ ]:
winners['age'].mean()

In [ ]:
new_df

In [ ]:
# get number of evaluations in each season

def num_episodes(row, evaluations):
    if row['won']:
        evaluations[row['seasonID']] = len(row['evaluations'])

evaluations = {}
for i, row in new_df.iterrows():
    num_episodes(row, evaluations)
print(evaluations)


def add_num_eval(row, evaluations):
    return evaluations[row['seasonID']]

new_df['numEval'] = new_df.apply(add_num_eval, args=(evaluations,), axis=1)
new_df



could use a column saying what place everyone took

In [ ]:
# Some facts about the winners
import numpy as np
import matplotlib.pyplot as plt

def eval_count(df):
    a = []
    for item in df['evaluations']:
        a = np.concatenate([a, item])
        
    values = np.unique(a, return_counts=True)

    values = pd.DataFrame({'counts':values[1], 'values': values[0]})
    return values
winner_values = eval_count(winners)
all_values = eval_count(new_df)

sns.barplot(data=winner_values, x='counts', y='values', hue='values', legend=None)
plt.show()
sns.barplot(data=all_values, x='counts', y='values', hue='values', legend=None)

In [ ]:
all_values

In [ ]:
def proportion(row, total_no_elim):
    if row['values'] in ['ELIM', 'WINNER', 'RUNNER_UP(S)']:
        return 0
    return row['counts']/total_no_elim

win_total = winner_values['counts'].sum() - winner_values[winner_values['values'].isin(['ELIM', 'WINNER', 'RUNNER_UP(S)'])]['counts'].sum()
all_total = all_values['counts'].sum() - all_values[all_values['values'].isin(['ELIM', 'WINNER', 'RUNNER_UP(S)'])]['counts'].sum()


all_values['proportion'] = all_values.apply(proportion, args = (all_total,), axis=1)
winner_values['proportion'] = winner_values.apply(proportion, args = (win_total,), axis=1)




In [ ]:
all_values

In [ ]:
winner_values
# doesn't have runner up, wdr, or ban
winner_values = pd.concat([winner_values, pd.DataFrame.from_dict({'counts':[0, 0, 0], 'values':['BAN', 'RUNNER_UP(S)', 'WDR'], 'proportion':[0,0,0]})])

In [ ]:
winner_values.sort_values(by='values')['proportion'] - all_values.sort_values(by='values')['proportion']
combined = pd.merge(winner_values, all_values, on='values', suffixes=['_win', '_all'])
combined['prop_diff'] = combined['proportion_win'] - combined['proportion_all']

In [ ]:
combined

In [ ]:
sns.barplot(data=combined, x='prop_diff', y='values', hue='values', legend=None)
plt.text(0.5, -.18, "<-Winners had less | Winners had more ->", ha='center', fontsize=10, transform=plt.gca().transAxes)
plt.title('Winner vs all contestants proportion differences')

The clearest indication of a winner is definitely performance in individual challenges. Winners had individual wins as 18% of their evaluations, while the average competitor (including winners) only had 10% of their evaluations as individual wins. This means winners were nearly twice as likely as all competitors to win challenges.

Interestingly, team performance was much less different. If anything, they performed slightly worse than agerage, with a slightly lower proportion of wins.

high_i_w
0.123167   0.091639

win i w
0.180352   0.100248

We want to predict who will win on after each new evaluation. We can do this a few ways:

1. Train a model for each unique number of people.
2. 